In [7]:
!pip3 install owlready2

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.


In [8]:
from owlready2 import *

onto = get_ontology("http://test.org/productkg")
onto.base_iri

'http://test.org/productkg#'

In [9]:
from owlready2 import *
import datetime

dul_onto = get_ontology("http://www.ease-crc.org/ont/DUL.owl").load()
dul = get_namespace("http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#")

onto = get_ontology("http://www.semanticweb.org/german_tourism_activities")
onto.imported_ontologies.append(dul_onto)

with onto:
    onto.metadata.comment.append("Authors: Giulio Billi, Cono Cirone")
    onto.metadata.comment.append("Domain: Activity recommendation and tourism in German cities.")

    class Activity(Thing):
        comment = ["A general concept for an activity of interest. Represents physical locations and events."]
        label = ["Activity"]
        is_a = [dul.Entity]

    class City(Thing):
        comment = ["A German city where an activity is located."]
        label = ["City"]
        is_a = [dul.PhysicalPlace]

    class BudgetTier(Thing):
        comment = ["A classification representing the cost of an Activity (Free, Low, Medium, High)."]
        label = ["Budget Tier"]
        is_a = [dul.Concept]

    class LocationSetting(Thing):
        comment = ["A classification of the environment (Indoor or Outdoor)."]
        label = ["Location Setting"]
        is_a = [dul.Concept]

    class OperatingHours(Thing):
        comment = ["A concept to model a set of opening/closing times."]
        label = ["Operating Hours"]
        is_a = [dul.Description]

    class DayOfWeek(Thing):
        comment = ["A specific day of the week."]
        label = ["Day of Week"]
        is_a = [dul.TimeInterval]

    class Duration(Thing):
        comment = ["The length of time a tour takes."]
        label = ["Duration"]
        is_a = [dul.TimeInterval]

    class Language(Thing):
        comment = ["A human language in which a tour is offered."]
        label = ["Language"]
        is_a = [dul.InformationObject]

    class MeetingPoint(Thing):
        comment = ["A physical location where a tour starts."]
        label = ["Meeting Point"]
        is_a = [dul.PhysicalPlace]

    class Tour(Activity):
        comment = ["A guided activity to visit the city. It is an event."]
        label = ["Tour"]
        is_a = [dul.Event]

    class PhysicalVenue(Activity):
        comment = ["A physical location with set operating hours."]
        label = ["Physical Venue"]
        is_a = [dul.PhysicalPlace]

    AllDisjoint([Tour, PhysicalVenue])

    class Museum(PhysicalVenue):
        comment = ["An institution displaying artifacts."]
        label = ["Museum"]
    class Park(PhysicalVenue):
        comment = ["A public green space or natural area."]
        label = ["Park"]
    class NightlifeVenue(PhysicalVenue):
        comment = ["A location for evening entertainment."]
        label = ["Nightlife Venue"]
    class Sight(PhysicalVenue):
        comment = ["A location or structure of historical interest."]
        label = ["Sight"]

    AllDisjoint([Museum, Park, NightlifeVenue, Sight])

    class MuseumType(Thing):
        comment = ["Category of museum (e.g., Art, History)."]
        label = ["Museum Type"]
        is_a = [dul.Concept]
    class ParkType(Thing):
        comment = ["Category of park (e.g., Botanical Garden)."]
        label = ["Park Type"]
        is_a = [dul.Concept]
    class ClubType(Thing):
        comment = ["Category of nightlife venue (e.g., Bar, Disco)."]
        label = ["Club Type"]
        is_a = [dul.Concept]
    class SightType(Thing):
        comment = ["Category of sight (e.g., Monument)."]
        label = ["Sight Type"]
        is_a = [dul.Concept]

    AllDisjoint([Activity, City, BudgetTier, LocationSetting, OperatingHours, 
                 DayOfWeek, Duration, Language, MeetingPoint])


    # =========================================================================
    # OBJECT PROPERTIES
    # =========================================================================
    
    # --- Attributes & Classifications ---
    # We link these to dul.isClassifiedBy because Budget and LocationSetting are Concepts
    
    class hasBudget(ObjectProperty, FunctionalProperty, AsymmetricProperty, IrreflexiveProperty):
        domain = [Activity]
        range = [BudgetTier]
        label = ["has budget"]
        comment = ["A relation connecting an activity to its cost classification."]
        is_a = [dul.isClassifiedBy] 

    class hasLocationSetting(ObjectProperty, FunctionalProperty, AsymmetricProperty, IrreflexiveProperty):
        domain = [Activity]
        range = [LocationSetting]
        label = ["has location setting"]
        comment = ["A relation connecting an activity to its location setting (Indoor vs. Outdoor)."]
        is_a = [dul.isClassifiedBy]

    # --- Spatial & Temporal ---
    
    class isInCity(ObjectProperty, FunctionalProperty, AsymmetricProperty, IrreflexiveProperty):
        domain = [Activity]
        range = [City]
        label = ["is in city"]
        comment = ["A relation connecting an activity to the city it is in."]
        is_a = [dul.hasLocation]

    class hasMeetingPoint(ObjectProperty, FunctionalProperty, AsymmetricProperty, IrreflexiveProperty):
        domain = [Tour]
        range = [MeetingPoint]
        label = ["has meeting point"]
        comment = ["A relation connecting a tour to its starting location."]
        is_a = [dul.hasLocation]

    class hasDuration(ObjectProperty, FunctionalProperty, AsymmetricProperty, IrreflexiveProperty):
        domain = [Tour]
        range = [Duration]
        label = ["has duration"]
        comment = ["A relation connecting a tour to its duration."]
        is_a = [dul.hasTimeInterval]

    # --- Descriptive & Operational ---

    class hasOperatingHours(ObjectProperty, AsymmetricProperty, IrreflexiveProperty):
        domain = [PhysicalVenue]
        range = [OperatingHours]
        label = ["has operating hours"]
        comment = ["A relation connecting a physical venue to its set of operating hours."]
        # The Venue is described by the OperatingHours definition
        is_a = [dul.isDescribedBy]

    class appliesToDay(ObjectProperty, AsymmetricProperty, IrreflexiveProperty):
        domain = [OperatingHours]
        range = [DayOfWeek]
        label = ["applies to day"]
        comment = ["A relation connecting an operating hours entry to a day of the week."]
        # The OperatingHours definition is valid for a specific TimeInterval
        is_a = [dul.hasTimeInterval]

    class hasLanguage(ObjectProperty, AsymmetricProperty, IrreflexiveProperty):
        domain = [Tour]
        range = [Language]
        label = ["has language"]
        comment = ["A relation connecting a tour to its language(s)."]
        is_a = [dul.associatedWith]

    # --- Specific Classification Properties ---
    # All these connect an Entity to a Concept (Type), so they are sub-properties of isClassifiedBy
    
    class hasMuseumType(ObjectProperty, FunctionalProperty, AsymmetricProperty, IrreflexiveProperty):
        domain = [Museum]
        range = [MuseumType]
        label = ["has museum type"]
        comment = ["A relation connecting a museum to its category."]
        is_a = [dul.isClassifiedBy]

    class hasParkType(ObjectProperty, FunctionalProperty, AsymmetricProperty, IrreflexiveProperty):
        domain = [Park]
        range = [ParkType]
        label = ["has park type"]
        comment = ["A relation connecting a park to its category."]
        is_a = [dul.isClassifiedBy]

    class hasClubType(ObjectProperty, FunctionalProperty, AsymmetricProperty, IrreflexiveProperty):
        domain = [NightlifeVenue]
        range = [ClubType]
        label = ["has club type"]
        comment = ["A relation connecting a nightlife venue to its category."]
        is_a = [dul.isClassifiedBy]

    class hasSightType(ObjectProperty, FunctionalProperty, AsymmetricProperty, IrreflexiveProperty):
        domain = [Sight]
        range = [SightType]
        label = ["has sight type"]
        comment = ["A relation connecting a sight to its category."]
        is_a = [dul.isClassifiedBy]
    # =========================================================================
    # DATA PROPERTIES
    # =========================================================================
    class opensAt(DataProperty, FunctionalProperty):
        domain = [OperatingHours]
        range = [datetime.time] 
        label = ["opens at"]
        comment = ["An attribute describing the opening time for an Operating Hours entry."]
        is_a = [dul.hasDataValue]

    class closesAt(DataProperty, FunctionalProperty):
        domain = [OperatingHours]
        range = [datetime.time] 
        label = ["closes at"]
        comment = ["An attribute describing the closing time for an Operating Hours entry."]
        is_a = [dul.hasDataValue]

# Save the ontology
onto.save(file="german_city_tourism_final_d2.owl", format="rdfxml")
print('Ontology created and saved')

Ontology created and saved
